In [1]:
import glob
import pathlib
from pathlib import Path
import sys
import os

In [2]:
basepath = Path("Y:/specialk/two_photon/behavior")

In [3]:
reference_tiff_removal = Path(basepath).glob("*\References\*.tif")

for file in reference_tiff_removal:
    file.unlink()
    print(file, "removed")

reference_dir_removal = Path(basepath).glob("*\References")

for directory in reference_dir_removal:
    directory.rmdir()
    print(directory, "removed")
    
    

Y:\specialk\two_photon\behavior\20210602_LHE011_plane0-034\References\20210602_LHE011_plane0-034-Ch2-16bit-Reference.tif removed
Y:\specialk\two_photon\behavior\20210602_LHE011_plane0-034\References\20210602_LHE011_plane0-034-Window1-Ch2-8bit-Reference.tif removed
Y:\specialk\two_photon\behavior\20210602_LHE012_plane0-036\References\20210602_LHE012_plane0-036-Ch2-16bit-Reference.tif removed
Y:\specialk\two_photon\behavior\20210602_LHE012_plane0-036\References\20210602_LHE012_plane0-036-Window1-Ch2-8bit-Reference.tif removed
Y:\specialk\two_photon\behavior\20210602_LHE013_plane0-037\References\20210602_LHE013_plane0-037-Ch2-16bit-Reference.tif removed
Y:\specialk\two_photon\behavior\20210602_LHE013_plane0-037\References\20210602_LHE013_plane0-037-Window1-Ch2-8bit-Reference.tif removed
Y:\specialk\two_photon\behavior\20210602_LHE014_plane0-038\References\20210602_LHE014_plane0-038-Ch2-16bit-Reference.tif removed
Y:\specialk\two_photon\behavior\20210602_LHE014_plane0-038\References\202106

In [4]:
rawpath = "raw"

session_generator = Path(basepath).glob("*")

for session in session_generator:
    session = session / rawpath
    if session.is_dir():
        print(session, "already exists!")
    else:
        print("Making directory", session)
        Path(session).mkdir()

session_generator = Path(basepath).glob("*")

for session in session_generator:
    files = [item for item in session.iterdir() if item.is_file()]
    raw_dir = [directory for directory in session.iterdir() if directory.is_dir()]
    
    for file in files:
        file.rename(Path.joinpath(raw_dir[0], file.name))
        print("moved", file.name)

Making directory Y:\specialk\two_photon\behavior\20210602_LHE011_plane0-034\raw
Making directory Y:\specialk\two_photon\behavior\20210602_LHE012_plane0-036\raw
Making directory Y:\specialk\two_photon\behavior\20210602_LHE013_plane0-037\raw
Making directory Y:\specialk\two_photon\behavior\20210602_LHE014_plane0-038\raw
Making directory Y:\specialk\two_photon\behavior\20210602_LHE015_plane0-039\raw
Making directory Y:\specialk\two_photon\behavior\20210602_LHE016_plane0-040\raw
Making directory Y:\specialk\two_photon\behavior\20210603_LHE011_plane0-042\raw
Making directory Y:\specialk\two_photon\behavior\20210603_LHE012_plane0-043\raw
Making directory Y:\specialk\two_photon\behavior\20210603_LHE013_plane0-044\raw
Making directory Y:\specialk\two_photon\behavior\20210603_LHE014_plane0-045\raw
Making directory Y:\specialk\two_photon\behavior\20210603_LHE015_plane0-046\raw
Making directory Y:\specialk\two_photon\behavior\20210603_LHE016_plane0-047\raw
Making directory Y:\specialk\two_photon\